In [3]:
from numpy import *

In [4]:
def loadDataSet(fileName):
    dataMat = []
    fr = open(fileName)
    
    for line in fr.readlines():
        curLine = line.strip().split('\t')
        fltLine = map(float , curLine)
        dataMat.append(fltLine)
    
    return dataMat

In [5]:
def binSplitDataSet(dataSet , feature , value):
    mat0 = dataSet[nonzero(dataSet[: , feature] > value)[0] , :]
    mat1 = dataSet[nonzero(dataSet[: , feature] <= value)[0] , :]
    
    return mat0 , mat1

In [14]:
def createTree(dataSet , leafType = regLeaf , errType = regErr , ops=(1 , 4)):
    feat , val = chooseBestSplit(dataSet , leafType , errType , ops)
    
    if feat == None:
        return val
    
    retTree = {}
    retTree['spInd'] = feat
    retTree['spVal'] = val
    
    lSet , rSet = binSplitDataSet(dataSet , feat , val)
    
    retTree['left'] = createTree(lSet , leafType , errType , ops)
    retTree['right'] = createTree(rSet , leafType , errType , ops)    
    
    return retTree

In [19]:
testMat = mat(eye(4))
testMat

matrix([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]])

In [20]:
mat0 , mat1 = binSplitDataSet(testMat , 1 , 0.5)

In [21]:
mat0

matrix([[0., 1., 0., 0.]])

In [22]:
mat1

matrix([[1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]])

In [7]:
def regLeaf(dataSet):
    return mean(dataSet[: , -1])

def regErr(dataSet):
    return var(dataSet[: , -1]) * shape(dataSet)[0]

In [13]:
def chooseBestSplit(dataSet , leafType = regLeaf , errType = regErr , ops=(1,4)):
    tolS = ops[0]
    tolN = ops[1]
    
    if len(set(dataSet[: , -1].T.tolist()[0])) == 1:
        return None , leafType(dataSet)
    
    m , n = shape(dataSet)
    S = errType(dataSet)
    
    bestS = inf
    bestIndex = 0
    bestValue = 0
    
    for featIndex in range(n-1): #最后一列是target
        #for splitVal in set(dataSet[: , featIndex]): #set直接去掉重复的feature取值
        for splitVal in set(dataSet[:,featIndex].T.A.tolist()[0]):
            mat0 , mat1 = binSplitDataSet(dataSet , featIndex , splitVal)
            
            if (shape(mat0)[0]<tolN) or (shape(mat1)[0]<tolN):
                continue
            
            newS = errType(mat0) + errType(mat1)
            if newS<bestS:
                bestIndex = featIndex
                bestValue = splitVal
                bestS = newS
    
    if (S-bestS)<tolS:
        return None , leafType(dataSet)
    
    mat0 , mat1 = binSplitDataSet(dataSet , bestIndex , bestValue)
    
    if (shape(mat0)[0]<tolN) or (shape(mat1)[0]<tolN):
        return None , leafType(dataSet)
    
    return bestIndex , bestValue

In [10]:
myDat = loadDataSet('../MLiA_SourceCode/Ch09/ex00.txt')
myMat = mat(myDat)

In [15]:
createTree(myMat)

{'left': 1.0180967672413792,
 'right': -0.04465028571428572,
 'spInd': 0,
 'spVal': 0.48813}

In [16]:
myDat1 = loadDataSet('../MLiA_SourceCode/Ch09/ex0.txt')
myMat1 = mat(myDat1)

createTree(myMat1)

{'left': {'left': {'left': 3.9871632,
   'right': 2.9836209534883724,
   'spInd': 1,
   'spVal': 0.797583},
  'right': 1.980035071428571,
  'spInd': 1,
  'spVal': 0.582002},
 'right': {'left': 1.0289583666666666,
  'right': -0.023838155555555553,
  'spInd': 1,
  'spVal': 0.197834},
 'spInd': 1,
 'spVal': 0.39435}

In [17]:
def isTree(obj):
    return (type(obj).__name__ == 'dict')

def getMean(tree):
    if isTree(tree['right']):
        tree['right'] = getMean(tree['right'])
    if isTree(tree['left']):
        tree['left'] = getMean(tree['left'])
    
    return (tree['left']+tree['right'])/2.0


In [31]:
#剪枝函数
'''
def prune(tree , testData):
    if shape(testData)[0] == 0:
        return getMean(tree)
    
    if (isTree(tree['right']) or isTree(tree['left'])):
        lSet , rSet = binSplitDataSet(testData , tree['spInd'] , tree['spVal'])
        
    if isTree(tree['left']):
        tree['left'] = prune(tree['left'] , lSet)
    
    if isTree(tree['right']):
        tree['right'] = prune(tree['right'] , rSet)
    
    if not isTree(tree['left']) and not isTree(tree['right']):
        lSet , rSet = binSplitDataSet(testData , tree['spInd'] , tree['spVal'])
        
        #errorNoMerge = sum(power(lSet[:,-1]-tree['left'] , 2)) + sum(power(rSet[:,-1]-tree['right'] , 2))
        errorNoMerge = sum(power(lSet[:,-1] - tree['left'],2)) + sum(power(rSet[:,-1] - tree['right'],2))
        treeMean = (tree['left'] + tree['right'])/2.0
        errorMerge = sum(power(testData[: , -1] - treeMean , 2))
        
        if errorMerge<errorNoMerge:
            print('merging')
        else:
            return tree
    else:
        return tree'''

In [35]:
def prune(tree, testData):
    if shape(testData)[0] == 0: 
        return getMean(tree)
    
    if (isTree(tree['right']) or isTree(tree['left'])):
        lSet, rSet = binSplitDataSet(testData, tree['spInd'], tree['spVal'])
        
    if isTree(tree['left']): 
        tree['left'] = prune(tree['left'], lSet)
        
    if isTree(tree['right']): 
        tree['right'] =  prune(tree['right'], rSet)
        
    if not isTree(tree['left']) and not isTree(tree['right']):
        lSet, rSet = binSplitDataSet(testData, tree['spInd'], tree['spVal'])
        
        errorNoMerge = sum(power(lSet[:,-1] - tree['left'],2)) + sum(power(rSet[:,-1] - tree['right'],2))
        treeMean = (tree['left']+tree['right'])/2.0
        errorMerge = sum(power(testData[:,-1] - treeMean,2))
        
        if errorMerge < errorNoMerge: 
            print('merging')
            return treeMean
        else: 
            return tree
        
    else: return tree

In [36]:
myDat2 = loadDataSet('../MLiA_SourceCode/Ch09/ex2.txt')
myMat2 = mat(myDat2)
myTree = createTree(myMat2)

In [37]:
myDatTest = loadDataSet('../MLiA_SourceCode/Ch09/ex2test.txt')
myMat2Test = mat(myDatTest)

In [38]:
prune(myTree , myMat2Test)

merging
merging
merging
merging
merging
merging
merging
merging
merging


{'left': {'left': {'left': {'left': 105.24862350000001,
    'right': 112.42895575000001,
    'spInd': 0,
    'spVal': 0.958512},
   'right': {'left': {'left': {'left': {'left': 87.3103875,
       'right': {'left': {'left': 96.452867,
         'right': {'left': 104.825409,
          'right': {'left': 95.181793,
           'right': 102.25234449999999,
           'spInd': 0,
           'spVal': 0.872883},
          'spInd': 0,
          'spVal': 0.892999},
         'spInd': 0,
         'spVal': 0.910975},
        'right': 95.27584316666666,
        'spInd': 0,
        'spVal': 0.85497},
       'spInd': 0,
       'spVal': 0.944221},
      'right': {'left': 81.110152,
       'right': 88.78449880000001,
       'spInd': 0,
       'spVal': 0.811602},
      'spInd': 0,
      'spVal': 0.833026},
     'right': 102.35780185714285,
     'spInd': 0,
     'spVal': 0.790312},
    'right': 78.08564325,
    'spInd': 0,
    'spVal': 0.759504},
   'spInd': 0,
   'spVal': 0.952833},
  'right': {'left': {'l

In [25]:
testMat[: , -1]

matrix([[0.],
        [0.],
        [0.],
        [1.]])

In [26]:
def regLeaf(data)

In [27]:
b

array([[1, 2, 3],
       [2, 4, 5]])

In [29]:
mean(b[: , -1])

4.0

### 模型树

In [42]:
def linearSolve(dataSet):
    m , n = shape(dataSet)
    
    X = mat(ones((m , n)))
    Y = mat(ones((m , 1)))
    
    X[: , 1:n] = dataSet[: , 0:n-1]
    Y = dataSet[: , -1]
    
    xTx = X.T * X
    
    if linalg.det(xTx) == 0:
        raise NameError('cannot do inverse')
        
    ws = xTx.I * (X.T*Y)
    
    return ws , X , Y

In [43]:
def modelLeaf(dataSet):
    ws , X , Y = linearSolve(dataSet)
    return ws

def modelErr(dataSet):
    ws , X , Y = linearSolve(dataSet)
    yHat = X * ws
    
    return sum(power(Y-yHat , 2))

In [44]:
myMat2 = mat(loadDataSet('../MLiA_SourceCode/Ch09/exp2.txt'))
createTree(myMat2 , modelLeaf , modelErr , (1 , 10))

{'left': matrix([[1.69855694e-03],
         [1.19647739e+01]]), 'right': matrix([[3.46877936],
         [1.18521743]]), 'spInd': 0, 'spVal': 0.285477}

In [54]:
def regTreeEval(model , inDat):
    return float(model)

def modelTreeEval(model, inDat):
    n = shape(inDat)[1]
    X = mat(ones((1,n+1)))
    X[:,1:n+1]=inDat
    return float(X*model)

In [55]:
'''def treeForeCast(tree , inData , modelEval = regTreeEval):
    if not isTree(tree):
        return modelEval(tree , inData)
    
    if inData[tree['spInd']] > tree['spVal']:
        if isTree(tree['left']):
            return treeForeCast(tree['left'] , inData , modelEval)
        else:
            return modelEval(tree['left'] , inData)
    else:
        if isTree(tree['right']):
            return treeForeCast(tree['right'] , inData , modelEval)
        else:
            return modelEval(tree['right'] , inData)'''
def treeForeCast(tree, inData, modelEval=regTreeEval):
    if not isTree(tree): return modelEval(tree, inData)
    if inData[tree['spInd']] > tree['spVal']:
        if isTree(tree['left']): return treeForeCast(tree['left'], inData, modelEval)
        else: return modelEval(tree['left'], inData)
    else:
        if isTree(tree['right']): return treeForeCast(tree['right'], inData, modelEval)
        else: return modelEval(tree['right'], inData)

def createForeCast(tree , testData , modelEval = regTreeEval):
    m = len(testData)
    yHat = mat(zeros((m , 1)))
    
    for i in range(m):
        yHat[i , 0] = treeForeCast(tree , mat(testData[i]) , modelEval)
    
    return yHat

In [50]:
trainMat = mat(loadDataSet('../MLiA_SourceCode/Ch09/bikeSpeedVsIq_train.txt'))
testMat = mat(loadDataSet('../MLiA_SourceCode/Ch09/bikeSpeedVsIq_test.txt'))


In [56]:
myTree = createTree(trainMat , ops=(1,20))
yHat = createForeCast(myTree , testMat[: , 0] , modelTreeEval)

TypeError: only size-1 arrays can be converted to Python scalars